In [10]:
import pandas as pd
# importing relevant libraries
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import SVD, KNNWithMeans, KNNBasic, KNNBaseline
# from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
# from surprise.model_selection import GridSearchCV
import numpy as np
from surprise import accuracy
from surprise import Dataset, Reader
from sklearn.pipeline import Pipeline
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# The following will improve the default style and the font sizes of the chart
sns.set_style("darkgrid")
matplotlib.rcParams["font.size"] = 14
matplotlib.rcParams["figure.figsize"] = (10, 6)
matplotlib.rcParams["figure.facecolor"] = "#00000000"

In [2]:
# Load the ratings data
ratings_df = pd.read_csv('ratings.csv')

# Display the first few rows of the dataset
print(ratings_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [3]:
ratings_df = ratings_df.drop(columns=['timestamp'], axis=1)

In [4]:

# Load the movies data
movies_df = pd.read_csv('movies.csv')

# Display the first few rows of the dataset
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# Merge ratings with movie titles
movie_ratings_df = pd.merge(ratings_df, movies_df, on="movieId")

# Display the first few rows of the merged dataset
movie_ratings_df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [6]:
genres_split = movie_ratings_df['genres'].str.split('|').str[0]

# concatenate the new columns with the movie_ratings_df
movie_ratings_df = pd.concat([movie_ratings_df, genres_split], axis = 1)

# lets drop the original column with the genres
movie_ratings_df = movie_ratings_df.drop(columns = ['genres'])
movie_ratings_df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [7]:
movie_analysis_df = pd.read_csv('cleaned_movie_ratings.csv')
movie_analysis_df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [8]:
# Define the Reader object to specify the rating scale
reader = Reader(rating_scale=(1, 5))

# Load the data into the surprise Dataset format
data = Dataset.load_from_df(movie_analysis_df[['userId', 'movieId', 'rating']], reader)

In [22]:
# Define the KNNBaseline algorithm with item-based similarity
knn_base = KNNBaseline(sim_options={'name': 'pearson', 'user_based': False})

# Create a pipeline
pipe = Pipeline([('knn', knn_base)])
pipe

Pipeline(steps=[('knn',
                 <surprise.prediction_algorithms.knns.KNNBaseline object at 0x000001E56B3905B0>)])

In [23]:
trainset = data.build_full_trainset()
trainset

In [24]:
# Build the full trainset
trainset = data.build_full_trainset()

# Fit the pipeline on the trainset
pipeline.fit(trainset)

TypeError: fit() takes 2 positional arguments but 3 were given